# Infer-9-Topic-Models : Latent Dirichlet Allocation (LDA)

**Serie** : Programmation Probabiliste avec Infer.NET (9/12)  
**Duree estimee** : 60 minutes  
**Prerequis** : Infer-8-Model-Selection

---

## Objectifs

- Comprendre le topic modeling et LDA
- Implementer la structure documents-topics-mots
- Utiliser les distributions Dirichlet pour les melanges
- Inferer les topics a partir d'un corpus

---

## Navigation

| Precedent | Suivant |
|-----------|--------|
| [Infer-8-Model-Selection](Infer-8-Model-Selection.ipynb) | [Infer-10-Crowdsourcing](Infer-10-Crowdsourcing.ipynb) |

---

## 1. Configuration

Nous preparons l'environnement pour le topic modeling avec LDA (Latent Dirichlet Allocation). Ce modele generatif decouvre automatiquement les themes latents dans un corpus de documents en utilisant des distributions Dirichlet comme priors conjugues.

In [1]:
#r "nuget: Microsoft.ML.Probabilistic"
#r "nuget: Microsoft.ML.Probabilistic.Compiler"

using Microsoft.ML.Probabilistic;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Utilities;
using Microsoft.ML.Probabilistic.Math;
using Microsoft.ML.Probabilistic.Models;
using Microsoft.ML.Probabilistic.Algorithms;
using Microsoft.ML.Probabilistic.Compiler;

Console.WriteLine("Infer.NET pret !");

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Microsoft.ML.Probabilistic, 0.4.2504.701 Microsoft.ML.Probabilistic.Compiler, 0.4.2504.701

Infer.NET pret !


## 2. Introduction au Topic Modeling

### Probleme

Etant donne un corpus de documents, decouvrir les **themes latents** (topics) et la composition de chaque document.

### Applications

- Organisation automatique de documents
- Recommandation de contenu
- Analyse de tendances
- Recherche semantique

### Representation Bag-of-Words

Un document est represente par le compte de chaque mot, ignorant l'ordre.

```
"Le chat mange la souris" -> {le: 1, chat: 1, mange: 1, la: 1, souris: 1}
```

## 3. Structure LDA

### Modele generatif

Pour chaque document d :
1. Tirer la distribution de topics : $\theta_d \sim \text{Dirichlet}(\alpha)$
2. Pour chaque mot w dans d :
   - Tirer un topic : $z \sim \text{Discrete}(\theta_d)$
   - Tirer un mot : $w \sim \text{Discrete}(\phi_z)$

Pour chaque topic k :
- $\phi_k \sim \text{Dirichlet}(\beta)$ : distribution sur le vocabulaire

### Schema

```
                alpha
                  |
                  v
              theta[d]     (distribution topics par document)
                  |
                  v
               z[d,n]      (topic du mot n dans doc d)
                  |
                  v
               w[d,n]  <-- phi[z]  <-- beta
          (mot observe)   (dist mots par topic)
```

## 4. Implementation LDA Simplifiee

In [2]:
// Donnees : corpus synthetique
// Vocabulaire : [sport, equipe, match, politique, election, vote, musique, concert, artiste]
string[] vocabulaire = { "sport", "equipe", "match", "politique", "election", "vote", "musique", "concert", "artiste" };
int vocabSize = vocabulaire.Length;
int numTopics = 3;  // Sport, Politique, Musique

// Documents (indices des mots)
int[][] documents = {
    new[] { 0, 1, 2, 0, 2 },           // Doc 1 : sport
    new[] { 3, 4, 5, 4, 3 },           // Doc 2 : politique
    new[] { 6, 7, 8, 7, 6 },           // Doc 3 : musique
    new[] { 0, 1, 3, 4, 0 },           // Doc 4 : sport + politique
    new[] { 6, 7, 0, 1, 8 },           // Doc 5 : musique + sport
    new[] { 2, 2, 1, 0, 2 },           // Doc 6 : sport
    new[] { 5, 4, 3, 5, 4 }            // Doc 7 : politique
};

int numDocs = documents.Length;

Console.WriteLine("=== Corpus ===");
for (int d = 0; d < numDocs; d++)
{
    string mots = string.Join(", ", documents[d].Select(i => vocabulaire[i]));
    Console.WriteLine($"Doc {d+1} : {mots}");
}

=== Corpus ===


Doc 1 : sport, equipe, match, sport, match


Doc 2 : politique, election, vote, election, politique


Doc 3 : musique, concert, artiste, concert, musique


Doc 4 : sport, equipe, politique, election, sport


Doc 5 : musique, concert, sport, equipe, artiste


Doc 6 : match, match, equipe, sport, match


Doc 7 : vote, election, politique, vote, election


In [3]:
// Modele LDA simplifie (pour un seul document)

// Prior sur les topics (symetrique)
double[] alphaPrior = Enumerable.Repeat(1.0, numTopics).ToArray();

// Prior sur les mots par topic (symetrique)
double[] betaPrior = Enumerable.Repeat(1.0, vocabSize).ToArray();

// Distribution des mots par topic (phi)
Range topicRange = new Range(numTopics).Named("topic");
Range vocabRange = new Range(vocabSize).Named("vocab");

VariableArray<Vector> phi = Variable.Array<Vector>(topicRange).Named("phi");
phi[topicRange] = Variable.Dirichlet(betaPrior).ForEach(topicRange);

Console.WriteLine("Variables LDA definies.");
Console.WriteLine($"  Nombre de topics : {numTopics}");
Console.WriteLine($"  Taille vocabulaire : {vocabSize}");

Variables LDA definies.


  Nombre de topics : 3


  Taille vocabulaire : 9


In [4]:
// Inference pour un document
int docIndex = 0;  // Premier document (sport)
int[] docWords = documents[docIndex];
int numWordsInDoc = docWords.Length;

// Distribution de topics pour ce document
Variable<Vector> theta = Variable.Dirichlet(alphaPrior).Named("theta");

Range wordRange = new Range(numWordsInDoc).Named("word");
VariableArray<int> wordObs = Variable.Array<int>(wordRange).Named("wordObs");
VariableArray<int> topicAssign = Variable.Array<int>(wordRange).Named("topicAssign");

// IMPORTANT: SetValueRange est necessaire pour utiliser Variable.Switch()
topicAssign.SetValueRange(topicRange);

using (Variable.ForEach(wordRange))
{
    topicAssign[wordRange] = Variable.Discrete(theta);
    using (Variable.Switch(topicAssign[wordRange]))
    {
        wordObs[wordRange] = Variable.Discrete(phi[topicAssign[wordRange]]);
    }
}

wordObs.ObservedValue = docWords;

InferenceEngine moteurLDA = new InferenceEngine(new VariationalMessagePassing());
moteurLDA.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Dirichlet thetaPost = moteurLDA.Infer<Dirichlet>(theta);

Console.WriteLine($"\n=== Inference pour Doc {docIndex + 1} ===");
Console.WriteLine($"Mots : {string.Join(", ", docWords.Select(i => vocabulaire[i]))}\n");
Console.WriteLine($"Distribution de topics (theta) :");
Vector thetaMean = thetaPost.GetMean();
for (int k = 0; k < numTopics; k++)
{
    Console.WriteLine($"  Topic {k+1} : {thetaMean[k]:F3}");
}

Compiling model...

done.


Iterating: 


.

.

...

Mots : sport, equipe, match, sport, match



Distribution de topics (theta) :


  Topic 1 : 0,333


  Topic 2 : 0,333


  Topic 3 : 0,333


## 5. LDA sur Corpus Complet

In [5]:
// Inference simplifiee document par document

Console.WriteLine("=== LDA sur corpus complet ===");
Console.WriteLine();

// Distribution des mots par topic (initialisation supervisee pour demo)
// Topic 0 : Sport (mots 0, 1, 2)
// Topic 1 : Politique (mots 3, 4, 5)
// Topic 2 : Musique (mots 6, 7, 8)

for (int d = 0; d < numDocs; d++)
{
    int[] dWords = documents[d];
    int nWords = dWords.Length;
    
    // Comptage simple des mots par categorie
    int countSport = dWords.Count(w => w <= 2);
    int countPolitique = dWords.Count(w => w >= 3 && w <= 5);
    int countMusique = dWords.Count(w => w >= 6);
    double total = countSport + countPolitique + countMusique + 0.001;
    
    Console.WriteLine($"Doc {d+1} : Sport={countSport/total:F2}, Politique={countPolitique/total:F2}, Musique={countMusique/total:F2}");
}

Console.WriteLine("\n(Proportions basees sur les mots observes)");

=== LDA sur corpus complet ===


Doc 1 : Sport=1,00, Politique=0,00, Musique=0,00


Doc 2 : Sport=0,00, Politique=1,00, Musique=0,00


Doc 3 : Sport=0,00, Politique=0,00, Musique=1,00


Doc 4 : Sport=0,60, Politique=0,40, Musique=0,00


Doc 5 : Sport=0,40, Politique=0,00, Musique=0,60


Doc 6 : Sport=1,00, Politique=0,00, Musique=0,00


Doc 7 : Sport=0,00, Politique=1,00, Musique=0,00



(Proportions basees sur les mots observes)


## 6. Visualisation des Topics

In [6]:
// Distribution phi (mots par topic) - simulee

double[,] phiSimule = {
    // Topic Sport
    { 0.30, 0.30, 0.30, 0.02, 0.02, 0.02, 0.02, 0.01, 0.01 },
    // Topic Politique
    { 0.02, 0.02, 0.02, 0.30, 0.30, 0.30, 0.02, 0.01, 0.01 },
    // Topic Musique
    { 0.02, 0.01, 0.01, 0.02, 0.02, 0.02, 0.30, 0.30, 0.30 }
};

string[] topicNames = { "Sport", "Politique", "Musique" };

Console.WriteLine("=== Distribution Mots par Topic (phi) ===");
Console.WriteLine();

for (int k = 0; k < numTopics; k++)
{
    Console.WriteLine($"Topic {k+1} ({topicNames[k]}) :");
    
    // Top mots
    var topMots = Enumerable.Range(0, vocabSize)
        .Select(v => (mot: vocabulaire[v], prob: phiSimule[k, v]))
        .OrderByDescending(x => x.prob)
        .Take(3);
    
    foreach (var (mot, prob) in topMots)
    {
        Console.WriteLine($"  {mot,-12} : {prob:F2}");
    }
    Console.WriteLine();
}

=== Distribution Mots par Topic (phi) ===


Topic 1 (Sport) :


  sport        : 0,30


  equipe       : 0,30


  match        : 0,30


...

  vote         : 0,30


Topic 3 (Musique) :


  musique      : 0,30


  concert      : 0,30


  artiste      : 0,30


## 7. Prediction sur Nouveaux Documents

In [7]:
// Prediction de topics pour un nouveau document

int[] nouveauDoc = { 0, 1, 6, 7, 0 };  // Sport + Musique
Console.WriteLine("=== Prediction Nouveau Document ===");
Console.WriteLine($"Mots : {string.Join(", ", nouveauDoc.Select(i => vocabulaire[i]))}");

// Calcul des vraisemblances par topic
double[] logLik = new double[numTopics];

for (int k = 0; k < numTopics; k++)
{
    logLik[k] = 0;
    foreach (int w in nouveauDoc)
    {
        logLik[k] += Math.Log(phiSimule[k, w] + 1e-10);
    }
}

// Normalisation (softmax)
double maxLogLik = logLik.Max();
double[] lik = logLik.Select(ll => Math.Exp(ll - maxLogLik)).ToArray();
double sumLik = lik.Sum();
double[] topicProbs = lik.Select(l => l / sumLik).ToArray();

Console.WriteLine("\nProbabilites de topics :");
for (int k = 0; k < numTopics; k++)
{
    Console.WriteLine($"  {topicNames[k],-12} : {topicProbs[k]:F3}");
}

=== Prediction Nouveau Document ===


Mots : sport, equipe, musique, concert, sport



Probabilites de topics :


  Sport        : 0,937


  Politique    : 0,000


  Musique      : 0,062


## 8. Extensions de LDA

### Variantes

| Modele | Description |
|--------|-------------|
| **HDP** (Hierarchical Dirichlet Process) | Nombre de topics appris automatiquement |
| **Correlated Topic Model** | Correlations entre topics |
| **Dynamic Topic Model** | Evolution des topics dans le temps |
| **Supervised LDA** | Avec labels de documents |

## 9. Exercice : Analyser un Corpus

### Enonce

Etendez le vocabulaire et ajoutez des documents pour creer un corpus plus realiste avec 4 topics.

In [8]:
// EXERCICE : Corpus etendu

string[] vocabEtendu = {
    // Sport (0-3)
    "football", "basketball", "tennis", "competition",
    // Tech (4-7)
    "ordinateur", "logiciel", "internet", "application",
    // Cuisine (8-11)
    "recette", "ingredient", "cuisson", "gastronomie",
    // Voyage (12-15)
    "hotel", "avion", "destination", "tourisme"
};

int[][] docsEtendus = {
    new[] { 0, 1, 2, 3, 0 },     // Sport
    new[] { 4, 5, 6, 7, 5 },     // Tech
    new[] { 8, 9, 10, 11, 9 },   // Cuisine
    new[] { 12, 13, 14, 15, 13 },// Voyage
    new[] { 0, 4, 5, 1, 6 },     // Sport + Tech
    new[] { 8, 12, 13, 10, 14 }, // Cuisine + Voyage
    new[] { 2, 3, 0, 1, 2 },     // Sport
    new[] { 6, 7, 4, 5, 7 }      // Tech
};

Console.WriteLine("=== Corpus Etendu (4 topics) ===");

for (int d = 0; d < docsEtendus.Length; d++)
{
    // Classification simple basee sur les indices de mots
    int sport = docsEtendus[d].Count(w => w <= 3);
    int tech = docsEtendus[d].Count(w => w >= 4 && w <= 7);
    int cuisine = docsEtendus[d].Count(w => w >= 8 && w <= 11);
    int voyage = docsEtendus[d].Count(w => w >= 12);
    
    string dominant;
    int max = new[] { sport, tech, cuisine, voyage }.Max();
    if (sport == max) dominant = "Sport";
    else if (tech == max) dominant = "Tech";
    else if (cuisine == max) dominant = "Cuisine";
    else dominant = "Voyage";
    
    string mots = string.Join(", ", docsEtendus[d].Select(i => vocabEtendu[i]));
    Console.WriteLine($"Doc {d+1} [{dominant,-8}] : {mots}");
}

=== Corpus Etendu (4 topics) ===


Doc 1 [Sport   ] : football, basketball, tennis, competition, football


Doc 2 [Tech    ] : ordinateur, logiciel, internet, application, logiciel


Doc 3 [Cuisine ] : recette, ingredient, cuisson, gastronomie, ingredient


Doc 4 [Voyage  ] : hotel, avion, destination, tourisme, avion


Doc 5 [Tech    ] : football, ordinateur, logiciel, basketball, internet


Doc 6 [Voyage  ] : recette, hotel, avion, cuisson, destination


Doc 7 [Sport   ] : tennis, competition, football, basketball, tennis


Doc 8 [Tech    ] : internet, application, ordinateur, logiciel, application


## 10. Resume

| Concept | Description |
|---------|-------------|
| **LDA** | Modele generatif pour documents |
| **Topic** | Distribution sur le vocabulaire |
| **Theta** | Distribution de topics par document |
| **Phi** | Distribution de mots par topic |
| **Dirichlet** | Prior conjugue pour distributions categoriques |

---

## Prochaine etape

Dans [Infer-10-Crowdsourcing](Infer-10-Crowdsourcing.ipynb), nous explorerons :

- L'agregation de labels de crowdsourcing
- La modelisation de la fiabilite des annotateurs
- Les modeles Community pour groupes d'annotateurs